In [ ]:
# 구글 드라이브 마운트
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
from gensim import corpora
from gensim.models import LdaModel, TfidfModel

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/2023년 10월-11월 프로젝트/토큰화 진행 및 불용어 처리/2022년 기사(완료).csv')
df.drop(columns=['Unnamed: 0'],inplace=True)
df.reset_index(drop=True, inplace=True)
df

,date,url,tokenize_title,tokenize_content
0,2022-01-02T06:03:00,https://n.news.naver.com/mnews/article/366/000...,"['우', '티', '타다', '연말', '공세', '찻잔', '속', '태풍', ...","['택시', '비', '할인', '이용자', '수', '성장', '정체', '카카오..."
1,2022-01-02T09:57:00,https://n.news.naver.com/mnews/article/138/000...,"['고난', '해', '겪은', '카카오', '모빌리티', '상생', '강화', '...","['카카오', '모빌리티', '기업', '공개', '재도전한', '올해', '문어'..."
2,2022-01-02T12:33:00,https://n.news.naver.com/mnews/article/138/000...,"['내일', '부터', '방역', '패스', '유효', '기간', '네이버', '카...","['내일', '부터', '코로나', '방역', '패스', '유효', '기간', '적..."
3,2022-01-02T19:31:00,https://n.news.naver.com/mnews/article/243/000...,"['네이버', '메타', '버스', '카카오', '플랫폼', '스타트업', '미래'...","['네이버', '카카오', '지난해', '스타트업', '곳', '투자', '기술',..."
4,2022-01-03T10:36:00,https://n.news.naver.com/mnews/article/092/000...,"['카카오', '이제', '팬', '시대', '한국', '주도할']","['한국', '산업', '한번', '도약', '기회', '유저', '사용자', '시..."
...,...,...,...,...
9579,2022-12-30T12:20:00,https://n.news.naver.com/mnews/article/031/000...,"['팬데믹', '부터', '엔데믹', '카카오', '내비', '결산', '데이터',...","['지난', '간', '이동', '트렌드', '분석', '해외', '여행', '수요..."
9580,2022-12-30T13:04:00,https://n.news.naver.com/mnews/article/119/000...,"['카카오', '장애', '소상', '공인', '보상', '최대', '여원', '어...","['카카오', '피해', '보상', '발표', '소상', '공인', '피해', '지..."
9581,2022-12-30T16:15:00,https://n.news.naver.com/mnews/article/015/000...,"['일반', '이용자', '카카오', '보상', '받는다', '내용', '봤더니']","['서랍', '플러스', '이용권', '보상', '지급', '전체', '이용자', ..."
9582,2022-12-30T17:07:00,https://n.news.naver.com/mnews/article/138/000...,"['외양간', '고치는', '카카오', '사과', '부터', '보상', '모범', ...","['카카오', '지난', '서비스', '장애', '발생', '이후', '보상', '..."


In [ ]:
# 각 'tokenize_title'과 'tokenize_content'의 type이 str이므로 이를 list로 변환하기 위함
def str_list(token):
  str_list = token
  str_list = str_list.replace("[","").replace("]","").replace("'","").replace(" ","")
  result = str_list.split(",")
  return result

In [ ]:
# title에 대한 LDA모델 적용
title_docs = []
for doc in df['tokenize_title']:
  title_docs.append(str_list(doc))
title_docs

[['우', '티', '타다', '연말', '공세', '찻잔', '속', '태풍', '카카오', '모방', '한계'],
 ['고난', '해', '겪은', '카카오', '모빌리티', '상생', '강화', '재도전'],
 ['내일', '부터', '방역', '패스', '유효', '기간', '네이버', '카카오', '앱', '업데이트'],
 ['네이버', '메타', '버스', '카카오', '플랫폼', '스타트업', '미래', '찾는', '빅', '테크'],
 ['카카오', '이제', '팬', '시대', '한국', '주도할'],
 ['네이버', '카카오', '새로운', '격전', '지는', '디지털', '헬'],
 ['개장', '시', '황', '새해', '첫', '코스피', '코스닥', '상승', '출발', '카겜', '위메이드'],
 ['시', '황', '코스피', '효과', '대감', '솔솔', '삼성', '전자', '카카오', '오름세'],
 ['신년사', '카카오', '팬', '시대', '왔다', '한국', '우위', '가질'],
 ['삼성', '전자', '이재용', '국내', '주식', '부호', '카카오', '김범수'],
 ['신년사', '조계현', '카카오', '게임', '대표', '비욘드', '게임', '보여줄', '해'],
 ['신년사', '카카오', '팬', '시대', '도래', '산업', '도약', '기회'],
 ['신년사', '조계현', '카카오', '게임', '대표', '올해', '비욘드', '게임', '보여주는', '해'],
 ['신년사', '조계현', '카카오', '게임', '대표', '비욘드', '게임', '본격', '적', '보여줄'],
 ['신년사', '조계현', '카카오', '게임', '대표', '비욘드', '게임', '진가', '보여줄'],
 ['조계현', '카카오', '게임', '대표', '메타', '버스', '스포츠', '적극', '확장'],
 ['조계현', '카카오', '게임', '비욘드', '게임', '확장', '박차'],


In [ ]:
id2word_title = corpora.Dictionary(title_docs)
print(id2word_title)

Dictionary<8333 unique tokens: ['공세', '모방', '속', '연말', '우']...>


In [ ]:
for value in id2word_title:
  print(value, id2word_title[value])

스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
3333 오염시킨
3334 페미니즘
3335 자신있어
3336 바꾸는
3337 였다
3338 기아
3339 몰려
3340 슈퍼
3341 돌아가겠다
3342 챙긴
3343 첫날
3344 보여준다
3345 박정호
3346 손잡다
3347 열었다
3348 젊어지는
3349 이끄는
3350 주주총회
3351 통과
3352 넘어선다
3353 조화
3354 갖춘
3355 보람
3356 함께한
3357 앱스토어
3358 청
3359 넘긴
3360 바톤
3361 갖게
3362 물러나는
3363 안정감
3364 르포
3365 아시나요
3366 오름
3367 제주도
3368 출항
3369 각자
3370 잇는다
3371 위원회
3372 미디어
3373 개막
3374 뭉쳤다
3375 용사
3376 열렸다
3377 올랐다
3378 바꿨다
3379 빼고
3380 달라질까
3381 뚜비
3382 망월
3383 수성구
3384 쏟아낸다
3385 어린이
3386 담화
3387 전통주
3388 대동
3389 이륜차
3390 전기
3391 총서
3392 부여
3393 드는
3394 얻을까
3395 결의
3396 섭
3397 진웅
3398 주류
3399 인방
3400 지배
3401 키웠다
3402 의결
3403 기업가
3404 된다며
3405 미니
3406 손댄다
3407 체질개선
3408 소수점
3409 담보
3410 국토부
3411 권한
3412 방통위
3413 비율
3414 네트워크
3415 샌드박스
3416 협업
3417 규합
3418 거버넌스
3419 리즘
3420 하이퍼
3421 왕좌
3422 새로
3423 포함
3424 경기도
3425 놀자
3426 야
3427 두기
3428 랩스
3429 케어
3430 그대로
3431 번호
3432 아이폰
3433 업뎃
3434 하자
3435 꿇은
3436 득실
3437 무릎
3438 단단해졌다
3439 지킴
3440 동거
3441 갔네
3442 뿐
3443 했을
3444 별
3445

In [ ]:
corpus_TDM = []
for doc in title_docs:
  result = id2word_title.doc2bow(doc)
  corpus_TDM.append(result)

corpus_TDM # index, 등장 횟수 (순서는 index 순으로 sorting해서 출력됨)

[[(0, 1),
  (1, 1),
  (2, 1),
  (3, 1),
  (4, 1),
  (5, 1),
  (6, 1),
  (7, 1),
  (8, 1),
  (9, 1),
  (10, 1)],
 [(6, 1), (11, 1), (12, 1), (13, 1), (14, 1), (15, 1), (16, 1), (17, 1)],
 [(6, 1),
  (18, 1),
  (19, 1),
  (20, 1),
  (21, 1),
  (22, 1),
  (23, 1),
  (24, 1),
  (25, 1),
  (26, 1)],
 [(6, 1),
  (20, 1),
  (27, 1),
  (28, 1),
  (29, 1),
  (30, 1),
  (31, 1),
  (32, 1),
  (33, 1),
  (34, 1)],
 [(6, 1), (35, 1), (36, 1), (37, 1), (38, 1), (39, 1)],
 [(6, 1), (20, 1), (40, 1), (41, 1), (42, 1), (43, 1), (44, 1)],
 [(45, 1),
  (46, 1),
  (47, 1),
  (48, 1),
  (49, 1),
  (50, 1),
  (51, 1),
  (52, 1),
  (53, 1),
  (54, 1),
  (55, 1)],
 [(6, 1),
  (48, 1),
  (54, 1),
  (55, 1),
  (56, 1),
  (57, 1),
  (58, 1),
  (59, 1),
  (60, 1),
  (61, 1)],
 [(6, 1), (35, 1), (38, 1), (39, 1), (62, 1), (63, 1), (64, 1), (65, 1)],
 [(6, 1), (57, 1), (60, 1), (66, 1), (67, 1), (68, 1), (69, 1), (70, 1)],
 [(6, 1), (17, 1), (63, 1), (71, 2), (72, 1), (73, 1), (74, 1), (75, 1)],
 [(6, 1), (35, 1), 

In [ ]:
# 토큰화 진행 및 불용어 처리에서 불용어에 대해 업데이트 해줘야함!!!
tfidf = TfidfModel(corpus_TDM)
corpus_TFIDF = tfidf[corpus_TDM]

n = 10 # 토픽갯수 - 변경가능
lda = LdaModel(corpus=corpus_TFIDF, id2word=id2word_title, num_topics=n, random_state=100)

for t in lda.print_topics():
  print(t[0],":",t[1])

0 : 0.032*"사태" + 0.028*"장애" + 0.026*"사과" + 0.024*"먹통" + 0.017*"대표" + 0.013*"서비스" + 0.007*"국감" + 0.007*"숙여" + 0.007*"사례" + 0.006*"책임"
1 : 0.065*"보상" + 0.017*"위반" + 0.016*"피해" + 0.013*"영업" + 0.012*"시스템" + 0.010*"기자회견" + 0.008*"증권" + 0.008*"속보" + 0.007*"이익" + 0.007*"먹통"
2 : 0.025*"대책" + 0.023*"관련" + 0.021*"정상화" + 0.015*"발표" + 0.013*"케이" + 0.007*"뱅크" + 0.007*"오늘" + 0.006*"계획" + 0.006*"불통" + 0.006*"규제"
3 : 0.042*"각자" + 0.022*"공인" + 0.021*"소상" + 0.018*"점검" + 0.014*"대표" + 0.011*"현장" + 0.011*"피해" + 0.009*"방지" + 0.009*"재발" + 0.009*"게임"
4 : 0.027*"데이터" + 0.025*"피해" + 0.025*"센터" + 0.021*"먹통" + 0.018*"화재" + 0.016*"서비스" + 0.014*"접수" + 0.013*"무료" + 0.011*"복구" + 0.011*"판교"
5 : 0.031*"숙인" + 0.018*"이용자" + 0.012*"협의" + 0.011*"대표" + 0.011*"김범수" + 0.010*"창업" + 0.010*"답변" + 0.008*"이니셔티브" + 0.008*"비트" + 0.008*"서버"
6 : 0.024*"방지법" + 0.024*"국감" + 0.022*"먹통" + 0.020*"사태" + 0.019*"국회" + 0.017*"김범수" + 0.017*"증인" + 0.015*"방위" + 0.014*"추진" + 0.010*"논의"
7 : 0.015*"대란" + 0.012*"대응" + 0.012*"특징" + 0.011*"대리" + 0.011*

In [ ]:
# 반복문 돌려서 기사에 토픽 일일이 배정해줘야 함(title 부분)
lda[corpus_TFIDF[0]]

[(0, 0.108061165),
 (1, 0.6115158),
 (2, 0.025535394),
 (3, 0.10169888),
 (4, 0.025531074),
 (5, 0.025531095),
 (6, 0.025531754),
 (7, 0.025531081),
 (8, 0.02553109),
 (9, 0.025532642)]

In [ ]:
for i in range(len(corpus_TFIDF)):
  print(lda[corpus_TFIDF[i]])

스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
[(0, 0.028855551), (1, 0.028836915), (2, 0.028838605), (3, 0.028838387), (4, 0.028837543), (5, 0.028836805), (6, 0.7404324), (7, 0.02883788), (8, 0.028847236), (9, 0.028838703)]
[(0, 0.29883188), (1, 0.027404225), (2, 0.02740706), (3, 0.027407771), (4, 0.027406333), (5, 0.027405426), (6, 0.48191625), (7, 0.027405895), (8, 0.027403848), (9, 0.027411357)]
[(0, 0.23210661), (1, 0.029093768), (2, 0.029098677), (3, 0.029099805), (4, 0.029094584), (5, 0.029096318), (6, 0.5351262), (7, 0.02909486), (8, 0.029094178), (9, 0.029095022)]
[(0, 0.29894394), (1, 0.027404219), (2, 0.027407052), (3, 0.027407764), (4, 0.027406327), (5, 0.027405418), (6, 0.48180422), (7, 0.02740589), (8, 0.027403845), (9, 0.027411342)]
[(0, 0.3260967), (1, 0.025456447), (2, 0.025458379), (3, 0.15617669), (4, 0.025458409), (5, 0.025458064), (6, 0.33952203), (7, 0.025457915), (8, 0.02545728), (9, 0.02545805)]
[(0, 0.27498057), (1, 0.02660532), (2, 0.16862048), (3, 0.026606956), (4, 0.02

In [ ]:
for i in range(len(corpus_TFIDF)):
  dominant_topic = sorted(lda[corpus_TFIDF[i]], key=lambda x: x[1], reverse=True)[0]
  df.loc[i, 'topic(title)'] = dominant_topic[0]
  df.loc[i, 'topic_details(title)'] = dominant_topic[1]
df

,date,url,tokenize_title,tokenize_content,topic(title),topic_details(title)
0,2022-01-02T06:03:00,https://n.news.naver.com/mnews/article/366/000...,"['우', '티', '타다', '연말', '공세', '찻잔', '속', '태풍', ...","['택시', '비', '할인', '이용자', '수', '성장', '정체', '카카오...",1.0,0.611523
1,2022-01-02T09:57:00,https://n.news.naver.com/mnews/article/138/000...,"['고난', '해', '겪은', '카카오', '모빌리티', '상생', '강화', '...","['카카오', '모빌리티', '기업', '공개', '재도전한', '올해', '문어'...",2.0,0.733601
2,2022-01-02T12:33:00,https://n.news.naver.com/mnews/article/138/000...,"['내일', '부터', '방역', '패스', '유효', '기간', '네이버', '카...","['내일', '부터', '코로나', '방역', '패스', '유효', '기간', '적...",1.0,0.661654
3,2022-01-02T19:31:00,https://n.news.naver.com/mnews/article/243/000...,"['네이버', '메타', '버스', '카카오', '플랫폼', '스타트업', '미래'...","['네이버', '카카오', '지난해', '스타트업', '곳', '투자', '기술',...",3.0,0.661517
4,2022-01-03T10:36:00,https://n.news.naver.com/mnews/article/092/000...,"['카카오', '이제', '팬', '시대', '한국', '주도할']","['한국', '산업', '한번', '도약', '기회', '유저', '사용자', '시...",1.0,0.581213
...,...,...,...,...,...,...
9579,2022-12-30T12:20:00,https://n.news.naver.com/mnews/article/031/000...,"['팬데믹', '부터', '엔데믹', '카카오', '내비', '결산', '데이터',...","['지난', '간', '이동', '트렌드', '분석', '해외', '여행', '수요...",1.0,0.564413
9580,2022-12-30T13:04:00,https://n.news.naver.com/mnews/article/119/000...,"['카카오', '장애', '소상', '공인', '보상', '최대', '여원', '어...","['카카오', '피해', '보상', '발표', '소상', '공인', '피해', '지...",3.0,0.351103
9581,2022-12-30T16:15:00,https://n.news.naver.com/mnews/article/015/000...,"['일반', '이용자', '카카오', '보상', '받는다', '내용', '봤더니']","['서랍', '플러스', '이용권', '보상', '지급', '전체', '이용자', ...",1.0,0.628441
9582,2022-12-30T17:07:00,https://n.news.naver.com/mnews/article/138/000...,"['외양간', '고치는', '카카오', '사과', '부터', '보상', '모범', ...","['카카오', '지난', '서비스', '장애', '발생', '이후', '보상', '...",8.0,0.680561


In [ ]:
# 토픽별 비중 확인
df['topic(title)'].unique()

array([1., 2., 3., 8., 7., 0., 5., 4., 6., 9.])

In [ ]:
df['topic(title)'].value_counts()

0.0    1458
4.0    1432
6.0    1140
9.0    1034
7.0    1005
3.0     923
5.0     737
1.0     717
2.0     572
8.0     566
Name: topic(title), dtype: int64

In [ ]:
# content에 대한 LDA모델 적용
content_docs = []
for doc in df['tokenize_content']:
  content_docs.append(str_list(doc))
content_docs

Output hidden; open in https://colab.research.google.com to view.

In [ ]:
id2word_content = corpora.Dictionary(content_docs)
print(id2word_content)

Dictionary<43444 unique tokens: ['가로막혀', '가맹', '가운데', '가장', '가파르']...>


In [ ]:
for value in id2word_content:
  print(value, id2word_content[value])

스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
38444 선로
38445 세세히
38446 쓰여졌다라는
38447 쓰였다
38448 아니다라는
38449 없어졌을
38450 올라가기는
38451 옮긴다고
38452 왔었고
38453 유용했냐는
38454 줄었는지
38455 토장
38456 파다
38457 회복하도록
38458 깨졌을까
38459 데을
38460 두되
38461 두었을
38462 밀접히
38463 안전하지만
38464 않았는가
38465 애매하게
38466 엄청나다
38467 여긴다
38468 연소
38469 왠만한
38470 이어가도록
38471 지었고
38472 톡임
38473 해하
38474 거둬들였다
38475 돌아서서
38476 빈번히
38477 추세라면
38478 적막
38479 내놓는가
38480 돌아왔을
38481 마시기
38482 만났고
38483 맞던
38484 여겨지
38485 풍랑
38486 휘하며
38487 가졌다고
38488 재야
38489 축전지
38490 일어서고
38491 파헤치겠다
38492 남았나요
38493 들려
38494 매만지고
38495 이뤄져야겠다는
38496 어떨지
38497 꽂아
38498 외산
38499 해보시죠
38500 가능한가요
38501 가연
38502 가우
38503 가우스
38504 가지고서
38505 가트너
38506 같기는
38507 거여
38508 건지는
38509 괜찮은지
38510 구어
38511 금품
38512 나가더라도
38513 나네요
38514 나누는데
38515 나오고는
38516 내연기관차
38517 넓지
38518 넘는다는
38519 놓죠
38520 닛산
38521 대단하네요
38522 대부터는
38523 되겠는데요
38524 되기로
38525 뒀고요
38526 드는군요
38527 드렸는데
38528 들으니까
38529 리즌
38530 마는
38531 만들어지다
38532 만들었고요
38533 많더라고요
38534 말씀드렸는데
38535 모라
385

In [ ]:
corpus_TDM = []
for doc in content_docs:
  result = id2word_content.doc2bow(doc)
  corpus_TDM.append(result)

corpus_TDM # index, 등장 횟수 (순서는 index 순으로 sorting해서 출력됨)

Output hidden; open in https://colab.research.google.com to view.

In [ ]:
# 토큰화 진행 및 불용어 처리에서 불용어에 대해 업데이트 해줘야함!!!
tfidf = TfidfModel(corpus_TDM)
corpus_TFIDF = tfidf[corpus_TDM]

n = 10 # 토픽갯수 - 변경가능
lda = LdaModel(corpus=corpus_TFIDF, id2word=id2word_content, num_topics=n, random_state=100)

for t in lda.print_topics():
  print(t[0],":",t[1])

0 : 0.002*"앞줄" + 0.002*"분가" + 0.001*"왼쪽" + 0.001*"과학" + 0.001*"이재용" + 0.001*"주식" + 0.000*"재산" + 0.000*"전해졌습니다" + 0.000*"총수" + 0.000*"회장"
1 : 0.006*"복구" + 0.005*"서비스" + 0.004*"방송통신" + 0.004*"장애" + 0.004*"게임" + 0.003*"데이터" + 0.003*"이용자" + 0.003*"로그인" + 0.002*"상공" + 0.002*"기능"
2 : 0.008*"기사" + 0.006*"심야" + 0.005*"분류" + 0.005*"섹션" + 0.005*"안내" + 0.001*"서구" + 0.001*"의회" + 0.001*"인천" + 0.000*"면세점" + 0.000*"매출"
3 : 0.006*"피해" + 0.004*"화재" + 0.004*"보상" + 0.004*"센터" + 0.004*"사태" + 0.004*"데이터" + 0.003*"먹통" + 0.003*"국회" + 0.003*"의원" + 0.003*"판교"
4 : 0.004*"보험" + 0.003*"공연" + 0.002*"기아" + 0.002*"강남" + 0.002*"주재" + 0.002*"보라" + 0.002*"구글" + 0.001*"짓고" + 0.001*"판매" + 0.001*"결제"
5 : 0.011*"대국민" + 0.010*"아지트" + 0.006*"택시" + 0.005*"기자회견" + 0.004*"방송통신" + 0.003*"호출" + 0.003*"열린" + 0.003*"전원" + 0.003*"모빌리티" + 0.003*"운전"
6 : 0.003*"신용" + 0.003*"프로필" + 0.002*"대출" + 0.002*"모빌리티" + 0.002*"캠퍼스" + 0.002*"분당구" + 0.002*"종로구" + 0.001*"서울역" + 0.001*"보험" + 0.001*"화재"
7 : 0.002*"무제" + 0.001*"적금" + 0.001*"원격" + 0.001

In [ ]:
# 반복문 돌려서 기사에 토픽 일일이 배정해줘야 함(content 부분)
lda[corpus_TFIDF[0]]

[(1, 0.7224501), (3, 0.21220665)]

In [ ]:
for i in range(len(corpus_TFIDF)):
  print(lda[corpus_TFIDF[i]])

스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
[(0, 0.016470956), (1, 0.016476845), (2, 0.016471498), (3, 0.25651693), (4, 0.016482852), (5, 0.016472584), (6, 0.6116839), (7, 0.016471472), (8, 0.016482072), (9, 0.016470896)]
[(0, 0.015509245), (1, 0.015602177), (2, 0.015509743), (3, 0.01568855), (4, 0.015520025), (5, 0.015511517), (6, 0.6386579), (7, 0.01550972), (8, 0.2369819), (9, 0.015509198)]
[(0, 0.015192188), (1, 0.015268541), (2, 0.015192555), (3, 0.015326707), (4, 0.01519984), (5, 0.015194185), (6, 0.5747022), (7, 0.015192642), (8, 0.303539), (9, 0.015192131)]
[(0, 0.015331019), (1, 0.015338692), (2, 0.015331395), (3, 0.10455431), (4, 0.015338872), (5, 0.015332791), (6, 0.5766387), (7, 0.015331469), (8, 0.21147168), (9, 0.015331044)]
[(0, 0.015306451), (1, 0.015313077), (2, 0.015306623), (3, 0.4392732), (4, 0.015310716), (5, 0.015307639), (6, 0.43824983), (7, 0.015306917), (8, 0.015319101), (9, 0.015306409)]
[(0, 0.015048409), (1, 0.015129123), (2, 0.015048498), (3, 0.5457935), (4, 0.0150

In [ ]:
for i in range(len(corpus_TFIDF)):
  dominant_topic = sorted(lda[corpus_TFIDF[i]], key=lambda x: x[1], reverse=True)[0]
  df.loc[i, 'topic(content)'] = dominant_topic[0]
  df.loc[i, 'topic_details(content)'] = dominant_topic[1]
df

,date,url,tokenize_title,tokenize_content,topic(title),topic_details(title),topic(content),topic_details(content)
0,2022-01-02T06:03:00,https://n.news.naver.com/mnews/article/366/000...,"['우', '티', '타다', '연말', '공세', '찻잔', '속', '태풍', ...","['택시', '비', '할인', '이용자', '수', '성장', '정체', '카카오...",1.0,0.611523,1.0,0.722424
1,2022-01-02T09:57:00,https://n.news.naver.com/mnews/article/138/000...,"['고난', '해', '겪은', '카카오', '모빌리티', '상생', '강화', '...","['카카오', '모빌리티', '기업', '공개', '재도전한', '올해', '문어'...",2.0,0.733601,1.0,0.577065
2,2022-01-02T12:33:00,https://n.news.naver.com/mnews/article/138/000...,"['내일', '부터', '방역', '패스', '유효', '기간', '네이버', '카...","['내일', '부터', '코로나', '방역', '패스', '유효', '기간', '적...",1.0,0.661654,1.0,0.556538
3,2022-01-02T19:31:00,https://n.news.naver.com/mnews/article/243/000...,"['네이버', '메타', '버스', '카카오', '플랫폼', '스타트업', '미래'...","['네이버', '카카오', '지난해', '스타트업', '곳', '투자', '기술',...",3.0,0.661517,3.0,0.557705
4,2022-01-03T10:36:00,https://n.news.naver.com/mnews/article/092/000...,"['카카오', '이제', '팬', '시대', '한국', '주도할']","['한국', '산업', '한번', '도약', '기회', '유저', '사용자', '시...",1.0,0.581213,1.0,0.573307
...,...,...,...,...,...,...,...,...
9579,2022-12-30T12:20:00,https://n.news.naver.com/mnews/article/031/000...,"['팬데믹', '부터', '엔데믹', '카카오', '내비', '결산', '데이터',...","['지난', '간', '이동', '트렌드', '분석', '해외', '여행', '수요...",1.0,0.564413,1.0,0.502860
9580,2022-12-30T13:04:00,https://n.news.naver.com/mnews/article/119/000...,"['카카오', '장애', '소상', '공인', '보상', '최대', '여원', '어...","['카카오', '피해', '보상', '발표', '소상', '공인', '피해', '지...",3.0,0.351103,3.0,0.576300
9581,2022-12-30T16:15:00,https://n.news.naver.com/mnews/article/015/000...,"['일반', '이용자', '카카오', '보상', '받는다', '내용', '봤더니']","['서랍', '플러스', '이용권', '보상', '지급', '전체', '이용자', ...",1.0,0.628441,1.0,0.502740
9582,2022-12-30T17:07:00,https://n.news.naver.com/mnews/article/138/000...,"['외양간', '고치는', '카카오', '사과', '부터', '보상', '모범', ...","['카카오', '지난', '서비스', '장애', '발생', '이후', '보상', '...",8.0,0.680561,3.0,0.691258


In [ ]:
# 토픽별 비중 확인
df['topic(content)'].unique()

array([1., 3., 5., 8., 4., 2., 7., 6., 0., 9.])

In [ ]:
df['topic(content)'].value_counts()

3.0    5967
1.0    2470
8.0     713
5.0     179
4.0      94
6.0      79
2.0      72
7.0       4
0.0       4
9.0       2
Name: topic(content), dtype: int64

In [ ]:
# CSV 파일로 저장
csv_filename = "news_2022.csv"
df.to_csv(csv_filename, index=False)